# Import Required Libraries
This cell imports all the Python packages and libraries needed for the analysis below.

In [11]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import geopandas as gpd
from shapely.geometry import Point, mapping
%matplotlib inline

import warnings
warnings.simplefilter(action="ignore")
pd.options.display.float_format = '{:20.2f}'.format
pd.options.mode.chained_assignment = None

import cityImage as ci
from cityImage.centrality import calculate_centrality, straightness_centrality, reach_centrality, weight_nodes


import seaborn as sns

import dill

from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

# Set up file paths and parameters
This cell defines file paths, city names, and other parameters used throughout the analysis.

In [ ]:
city_names = [
    'London',
    'Birmingham',
    'Liverpool',
    'Bristol',
    'Manchester',
    'Sheffield',
    'Leeds',
    'Milton Keynes',
    'York',
    'Coventry',
    'Newcastle upon Tyne',
    'Norwich',
    'Cambridge'
    ]

places = [
    'London',
    'Birmingham',
    'Liverpool',
    'Bristol',
    'Manchester',
    'Sheffield',
    'Leeds',
    'Milton Keynes',
    'York',
    'Coventry',
    'Newcastle upon Tyne',
    'Norwich',
    'Cambridge'
    ]

epsg = 25832
crs = 'EPSG:'+str(epsg)
download_method = 'OSMplace'
distance = None

In [ ]:
# city_name = 'Leeds'
# epsg = 25832
# crs = 'EPSG:'+str(epsg)
# place = 'Leeds'
# download_method = 'OSMplace'
# distance = None

# Load and prepare network data
This cell loads the network data and prepares it for further analysis, such as cleaning or extracting features.

In [20]:
import osmnx as ox

for city in city_names:
    try:
        gdf = ox.geocode_to_gdf(city)
        print(f"Valid OSM place: {city}")
    except Exception as e:
        print(f"Invalid OSM place: {city} ({e})")

Valid OSM place: Milton Keynes
Valid OSM place: York
Valid OSM place: Coventry
Valid OSM place: Newcastle upon Tyne
Valid OSM place: Norwich
Valid OSM place: Cambridge


In [ ]:
import os

output_dir = "city_graphs"
os.makedirs(output_dir, exist_ok=True)

for city in city_names:
    city_underscore = city.replace(" ", "_")
    nodes, edges = ci.get_network_fromOSM(city, download_method, 'walk', epsg, distance=distance)
    nodes, edges = ci.clean_network(
        nodes,
        edges,
        dead_ends=False,
        remove_islands=True,
        self_loops=True,
        same_vertexes_edges=True
    )
    globals()[f"nodes_graph_{city_underscore}"] = nodes
    globals()[f"edges_graph_{city_underscore}"] = edges
    nodes.to_file(f"{output_dir}/nodes_graph_{city_underscore}.gpkg", driver="GPKG")
    edges.to_file(f"{output_dir}/edges_graph_{city_underscore}.gpkg", driver="GPKG")
    del nodes, edges, city, city_underscore

# 3.5hrs for all

In [ ]:
# import os

# output_dir = "city_graphs"
# os.makedirs(output_dir, exist_ok=True)

# for city in city_names:
#     nodes = globals()[f"nodes_graph_{city}"]
#     edges = globals()[f"edges_graph_{city}"]
#     nodes.to_file(f"{output_dir}/nodes_graph_{city}.gpkg", driver="GPKG")
#     edges.to_file(f"{output_dir}/edges_graph_{city}.gpkg", driver="GPKG")

### Create the dual_graph

In [ ]:
import os

output_dir = "city_dual_graphs"
os.makedirs(output_dir, exist_ok=True)

for city in city_names:
    city_underscore = city.replace(" ", "_")
    nodes_graph = gpd.read_file(f"city_graphs/nodes_graph_{city_underscore}.gpkg")
    edges_graph = gpd.read_file(f"city_graphs/edges_graph_{city_underscore}.gpkg")
    nodesDual_graph, edgesDual_graph = ci.dual_gdf(nodes_graph, edges_graph, epsg)
    globals()[f"nodesDual_graph_{city_underscore}"] = nodesDual_graph
    globals()[f"edgesDual_graph_{city_underscore}"] = edgesDual_graph
    nodesDual_graph.to_file(f"{output_dir}/nodesDual_graph_{city_underscore}.gpkg", driver="GPKG")
    edgesDual_graph.to_file(f"{output_dir}/edgesDual_graph_{city_underscore}.gpkg", driver="GPKG")
    del nodesDual_graph, edgesDual_graph, nodes_graph, edges_graph, city_underscore

# 2hrs total

In [ ]:
import dill
import os

output_dir = "city_dual_graphs"
os.makedirs(output_dir, exist_ok=True)

for city in city_names:
    city_underscore = city.replace(" ", "_")
    nodesDual_graph = gpd.read_file(f"city_dual_graphs/nodesDual_graph_{city_underscore}.gpkg")
    edgesDual_graph = gpd.read_file(f"city_dual_graphs/edgesDual_graph_{city_underscore}.gpkg")
    dual_graph = ci.dual_graph_fromGDF(nodesDual_graph, edgesDual_graph)
    with open(f"{output_dir}/dual_graph_{city_underscore}.dill", "wb") as f:
        dill.dump(dual_graph, f)
    del nodesDual_graph, edgesDual_graph, dual_graph, city_underscore

### Create the planar graph

In [ ]:
import dill
import os

output_dir = "city_graphs"
os.makedirs(output_dir, exist_ok=True)

for city in city_names:
    city_underscore = city.replace(" ", "_")
    nodes_graph = gpd.read_file(f"city_graphs/nodes_graph_{city_underscore}.gpkg")
    edges_graph = gpd.read_file(f"city_graphs/edges_graph_{city_underscore}.gpkg")
    graph = ci.graph_fromGDF(nodes_graph, edges_graph)
    with open(f"{output_dir}/graph_{city_underscore}.dill", "wb") as f:
        dill.dump(graph, f)
    del nodes_graph, edges_graph, graph, city_underscore

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(dpi=300, figsize=(10,10))
# fig = ci.plot_gdf(
#     edges_graph,
#     title = city_names + ': Street Network - walk',
#     black_background = False,
#     geometry_size = 0.1
# )

---

In [ ]:
# del nodes_graph_Sheffield, nodes_graph_Manchester, nodes_graph_London, nodes_graph_Liverpool, nodes_graph_Leeds, nodes_graph_Bristol, nodes_graph_Birmingham
# del nodesDual_graph_Sheffield, nodesDual_graph_Manchester, nodesDual_graph_London, nodesDual_graph_Liverpool, nodesDual_graph_Leeds, nodesDual_graph_Bristol, nodesDual_graph_Birmingham
# del nodes_graph, nodesDual_graph, nodes
# del edges_graph_Sheffield, edges_graph_Manchester, edges_graph_London, edges_graph_Liverpool, edges_graph_Leeds, edges_graph_Bristol, edges_graph_Birmingham
# del edgesDual_graph_Sheffield, edgesDual_graph_Manchester, edgesDual_graph_London, edgesDual_graph_Liverpool, edgesDual_graph_Leeds, edgesDual_graph_Bristol, edgesDual_graph_Birmingham
# del edgesDual_graph, edges_graph, edges
# del dual_graph, districts, graph

In [ ]:
# Delete a variable whose name contains a space
var_name = "edgesDual_graph_Newcastle upon Tyne"

if var_name in globals():
    del globals()[var_name]

list(globals().keys())


### Partitions

#### Partition each city dual_graph

In [54]:
import geopandas as gpd
import dill
import os

output_dir = "city_partitions"
os.makedirs(output_dir, exist_ok=True)

weights = ['deg']

for city in city_names:
    city_underscore = city.replace(" ", "_")
    edges_graph = gpd.read_file(f"city_graphs/edges_graph_{city_underscore}.gpkg")
    with open(f"city_dual_graphs/dual_graph_{city_underscore}.dill", "rb") as f:
        dual_graph = dill.load(f)
    districts = edges_graph.copy()
    for weight in weights:
        districts = ci.identify_regions(dual_graph, districts, weight=weight)
    districts.to_file(f"{output_dir}/districts_{city_underscore}.gpkg", driver="GPKG")
    globals()[f"districts_{city_underscore}"] = districts  # Keep in globals
    del edges_graph, dual_graph, city_underscore
del districts

### Plot some?

In [ ]:
columns = ['p_deg']
titles = ['Partition - Distance', 'Partition - Angles', 'Partition - Topological']

nlabels = max([len(districts[column].unique()) for column in columns])
cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
fig = ci.plot_grid_gdf_columns(districts, columns = columns, titles = titles, geometry_size = 1.5, cmap = cmap, black_background = True,
                  legend = False, figsize = (15, 10), ncols = 3, nrows = 1)

---

### Betweenness centrality

In [ ]:
ci.straightness_centrality()

# ?

In [ ]:
# Obtaining the graph from the case-study area and computing the centrality measures
graph = ci.graph_fromGDF(nodes_graph, edges_graph)
# betweenness centrality
Bc_Rd = ci.calculate_centrality(nx_graph = graph, measure='betweenness', weight='length')
# straightness centrality
Sc = ci.calculate_centrality(nx_graph = graph, measure='straightness', weight='length', normalized = True)

# ?

### Degree and Average node_degree for each edge

In [ ]:
# Calculate node degree using the networkx graph from cityImage
if hasattr(nodes_graph, 'graph') and isinstance(nodes_graph.graph, nx.Graph):
    G = nodes_graph.graph
else:
    # If not, try to reconstruct from edges_graph
    G = nx.from_pandas_edgelist(edges_graph, 'u', 'v')

# Map node degrees to nodes_graph index
nodes_graph['degree'] = nodes_graph.index.map(dict(G.degree()))
nodes_graph.head(1)

# 1 second

#### Node degree

In [ ]:
# Ensure nodes_graph has a 'degree' column (already calculated previously)
# nodes_graph['degree'] = nodes_graph.index.map(dict(graph.degree()))

# Map node degrees to edges
def get_edge_node_degrees(row, nodes_graph):
    u, v = row['u'], row['v']  # edge endpoints
    deg_u = nodes_graph.loc[u, 'degree']
    deg_v = nodes_graph.loc[v, 'degree']
    return (deg_u + deg_v) / 2  # mean degree

# If 'u' and 'v' columns exist in edges_graph:
edges_graph['mean_node_degree'] = edges_graph.apply(
    lambda row: get_edge_node_degrees(row, nodes_graph), axis=1
)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)

edges_graph.head(1)
# 2.2seconds

### Dead-end Density

In [ ]:
buffer_distance = 900  # meters

# 1. Identify dead-end nodes (degree == 1)
dead_end_nodes = nodes_graph[nodes_graph['degree'] == 1].copy()

# 2. Create buffers for all edges
edges_graph['buffer'] = edges_graph.geometry.buffer(buffer_distance)

# 3. Prepare a GeoDataFrame of buffers
buffers_gdf = edges_graph[['buffer']].copy()
buffers_gdf = buffers_gdf.set_geometry('buffer')
buffers_gdf['buffer_id'] = buffers_gdf.index

# 4. Spatial join: which dead ends fall in which buffer
joined = gpd.sjoin(
    dead_end_nodes[['geometry']],  # dead-end points
    buffers_gdf,                   # buffers
    how='left',
    predicate='within'
)

# 5. Count dead ends for each buffer (group by buffer index)
dead_end_counts = joined.groupby('buffer_id').size()

# 6. Calculate buffer areas
buffer_areas = buffers_gdf['buffer'].area

# 7. Assign dead ends density (per km²)
edges_graph['dead_end_density'] = (dead_end_counts / buffer_areas) * 1e6
edges_graph['dead_end_density'] = edges_graph['dead_end_density'].fillna(0)

# 8. Clean up
edges_graph.drop(columns=['buffer'], inplace=True)
del buffers_gdf, joined, dead_end_counts, buffer_areas, dead_end_nodes

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)

edges_graph.head(2)
# 33 seconds

#### Intersection density

In [ ]:
# Create a NetworkX graph from the edges_graph DataFrame
import networkx as nx
graph = nx.from_pandas_edgelist(edges_graph, 'u', 'v')

In [ ]:
# Calculate node degrees from the NetworkX graph
degree_dict = dict(graph.degree())

# Add the degree as a new column in nodes_graph
nodes_graph['degree'] = nodes_graph.index.map(degree_dict)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/nodes_graph.dill', 'wb') as file:
#     dill.dump(nodes_graph, file)

nodes_graph.head(2)
# 1 second

In [ ]:
buffer_distance = 900  # meters

# 1. Identify intersection nodes (degree >= 3)
nodes_graph['degree'] = nodes_graph['degree'] if 'degree' in nodes_graph else nodes_graph.degree
intersection_nodes = nodes_graph[nodes_graph['degree'] >= 3].copy()

# 2. Create buffers for all edges
edges_graph['buffer'] = edges_graph.geometry.buffer(buffer_distance)

# 3. Prepare a GeoDataFrame of buffers
buffers_gdf = edges_graph[['buffer']].copy()
buffers_gdf = buffers_gdf.set_geometry('buffer')
buffers_gdf['buffer_id'] = buffers_gdf.index

# 4. Spatial join: which intersections fall in which buffer
joined = gpd.sjoin(
    intersection_nodes[['geometry']],  # intersection points
    buffers_gdf,                       # buffers
    how='left',
    predicate='within'
)

# 5. Count intersections for each buffer (group by buffer index)
intersection_counts = joined.groupby('buffer_id').size()

# 6. Calculate buffer areas
buffer_areas = buffers_gdf.to_crs(epsg=3857)['buffer'].area

# 7. Assign intersection density (per km²)
edges_graph['intersection_density'] = (intersection_counts / buffer_areas) * 1e6
edges_graph['intersection_density'] = edges_graph['intersection_density'].fillna(0)

# 8. Clean up
edges_graph.drop(columns=['buffer'], inplace=True)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)

# del buffers_gdf, joined, intersection_counts, buffer_areas

# districts = districts.merge(
#     edges_graph[['edge_ID', 'intersection_density']],
#     how='left',
#     on='edge_ID'  # Adjust this column name based on your dataset
# )
edges_graph.head(2)
# 2.5 minutes

### Edge endpoint degree variance

In [ ]:
def edge_degree_variance(row, nodes_graph):
    u, v = row['u'], row['v']
    deg_u = nodes_graph.loc[u, 'degree']
    deg_v = nodes_graph.loc[v, 'degree']
    return np.var([deg_u, deg_v])

edges_graph['edge_degree_variance'] = edges_graph.apply(
    lambda row: edge_degree_variance(row, nodes_graph), axis=1
)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)
edges_graph.head(2)
# 4 seconds

#### 6. Circuity

In [ ]:
def calculate_circuity(row, nodes_graph):
    u, v = row['u'], row['v']
    # Get endpoint coordinates
    point_u = nodes_graph.loc[u, 'geometry']
    point_v = nodes_graph.loc[v, 'geometry']
    # Euclidean distance
    euclidean_dist = point_u.distance(point_v)
    # Actual edge length
    actual_length = row['length']
    # Avoid division by sero
    if euclidean_dist == 0:
        return 1  # or np.nan
    return actual_length / euclidean_dist

edges_graph['circuity'] = edges_graph.apply(
    lambda row: calculate_circuity(row, nodes_graph), axis=1
)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)
edges_graph.head(2)
# 3 seconds

#### 7. Orientation and orientation entropy

In [ ]:
# Calculate the abgles for all the edges
from scipy.stats import entropy
def edge_orientation(line):
    x0, y0, x1, y1 = *line.coords[0], *line.coords[-1]
    angle = np.degrees(np.arctan2(y1 - y0, x1 - x0)) % 180
    return angle

edges_graph['orientation'] = edges_graph.geometry.apply(edge_orientation)

# Calculate the angle entropy

def orientation_entropy_for_edge(row, edges_graph, buffer_distance=900, bins=18):
    buffer = row.geometry.buffer(buffer_distance)
    intersecting = edges_graph[edges_graph.geometry.intersects(buffer)]
    angles = intersecting['orientation']  # Use precomputed angles
    hist, _ = np.histogram(angles, bins=bins, range=(0, 180), density=True)
    hist += 1e-12
    return entropy(hist)

edges_graph['orientation_entropy'] = edges_graph.apply(
    lambda row: orientation_entropy_for_edge(row, edges_graph), axis=1
)

# # Save the updated GeoDataFrame using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)
edges_graph.head(2)

# 41 minutes

#### Node constraint

In [ ]:
# 1. Calculate node constraint for all nodes
constraint_dict = nx.constraint(graph)

# 2. Add constraint as a column to nodes_graph
nodes_graph['constraint'] = nodes_graph.index.map(constraint_dict)

# 3. Assign mean constraint of endpoints to each edge
def get_edge_node_constraint(row, nodes_graph):
    u, v = row['u'], row['v']
    c_u = nodes_graph.loc[u, 'constraint']
    c_v = nodes_graph.loc[v, 'constraint']
    return (c_u + c_v) / 2

edges_graph['mean_node_constraint'] = edges_graph.apply(
    lambda row: get_edge_node_constraint(row, nodes_graph), axis=1
)

# # Save the updated GeoDataFrames using dill
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/nodes_graph.dill', 'wb') as file:
#     dill.dump(nodes_graph, file)
# with open('C:/Users/gy21jw/Documents/Coding/CDAS_Dissertation/python_files/edges_graph.dill', 'wb') as file:
#     dill.dump(edges_graph, file)
edges_graph.head(2)

# 15 seconds

### Edge Density

In [ ]:
# Calculate edge density for each edge's buffer area (per km²)
buffer_distance = 900  # meters (same as other density measures)

# 1. Create buffers for all edges
edges_graph['buffer'] = edges_graph.geometry.buffer(buffer_distance)

# 2. Calculate buffer areas in projected CRS (meters)
buffers_gdf = edges_graph[['buffer']].copy()
buffers_gdf = buffers_gdf.set_geometry('buffer')
buffers_gdf['buffer_id'] = buffers_gdf.index
buffers_gdf = buffers_gdf.to_crs(epsg=3857)  # Project to meters for area calculation
buffer_areas = buffers_gdf['buffer'].area

# 3. Count number of edges intersecting each buffer
# (self-intersection included, so density is at least 1)
def count_edges_in_buffer(row, edges_gdf):
    buffer_geom = row['buffer']
    return edges_gdf.geometry.intersects(buffer_geom).sum()

edges_graph['edge_count_in_buffer'] = edges_graph.apply(lambda row: count_edges_in_buffer(row, edges_graph), axis=1)

# 4. Calculate edge density (edges per km²)
edges_graph['edge_density'] = (edges_graph['edge_count_in_buffer'] / buffer_areas) * 1e6
edges_graph['edge_density'] = edges_graph['edge_density'].fillna(0)

# 5. Clean up temporary columns
edges_graph.drop(columns=['buffer', 'edge_count_in_buffer'], inplace=True)
del buffers_gdf, buffer_areas
edges_graph.head(2)

### SAVE THE GRAPHS DATA

In [ ]:
with open('C:/Users/gy21jw/Coding/transfer_resubmission/nodesDual_graph.dill', 'wb') as file:
    dill.dump(nodesDual_graph, file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/edgesDual_graph.dill', 'wb') as file:
    dill.dump(edgesDual_graph, file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/nodes_graph.dill', 'wb') as file:
    dill.dump(nodes_graph, file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/edges_graph.dill', 'wb') as file:
    dill.dump(edges_graph, file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/dual_graph.dill', 'wb') as file:
    dill.dump(dual_graph, file)

### Districts

Now that network measures have been calculated - use cityImage to identify districts (regions) in the network based on the 'rad' value

In [2]:
# load the edges_graph, nodes_graph and the nodesDual_graph and the edgesDual_graph
with open('C:/Users/gy21jw/Coding/transfer_resubmission/edges_graph.dill', 'rb') as file:
    edges_graph = dill.load(file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/nodes_graph.dill', 'rb') as file:
    nodes_graph = dill.load(file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/edgesDual_graph.dill', 'rb') as file:
    edgesDual_graph = dill.load(file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/nodesDual_graph.dill', 'rb') as file:
    nodesDual_graph = dill.load(file)
with open('C:/Users/gy21jw/Coding/transfer_resubmission/dual_graph.dill', 'rb') as file:
    dual_graph = dill.load(file)

In [7]:
# Uncomment the weights (or as many as required) - USE 'rad'
weights = [
    'deg'
    ]
districts = edges_graph.copy()
for weight in weights:
    districts = ci.identify_regions(
        dual_graph=dual_graph,
        edges_gdf=districts,
        weight = weight
        )
del weight
partition_counts = districts['p_deg'].value_counts().sort_values(ascending=False)
print(f"Found {len(partition_counts)} distinct regions")

Found 254 distinct regions


In [ ]:
# rename the last column in districts back to 'p_rad'
# districts.columns = [*districts.columns[:-1], 'p_rad']

# Check how many partitions were created
# partition_counts = districts['p_rad'].value_counts().sort_values(ascending=False)
# print(f"Found {len(partition_counts)} distinct regions")
# # Check sizes of partitions
# print(f"Largest region has {partition_counts.iloc[0]} edges ({partition_counts.iloc[0]/len(districts)*100:.1f}%)")
# print(f"Smallest region has {partition_counts.iloc[-1]} edges ({partition_counts.iloc[-1]/len(districts)*100:.1f}%)")
# print(f"Mean region size: {partition_counts.mean():.1f} edges")
# print(f"Median region size: {partition_counts.median():.1f} edges")

# # Distribution of top regions
# print("\nTop 10 regions by size:")
# for i, (region, count) in enumerate(partition_counts.head(10).items()):
#     print(f"  Region {region}: {count} edges ({count/len(districts)*100:.1f}%)")

In [ ]:
import matplotlib.pyplot as plt


columns = ['p_deg']
titles = [f'CityImage Partitions by Degree (n={len(partition_counts)})']

nlabels = max([len(districts[column].unique()) for column in columns])

cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
plt.rcParams['figure.dpi'] = 300
fig = plt.figure(figsize=(10, 8), dpi=150)
fig = ci.plot_gdf(
    districts,
    column = columns[0],
    title = titles[0],
    geometry_size = 0.2,
    cmap = cmap,
    black_background = False,
    legend = False,
    figsize = (10, 8)
    )


In [184]:
with open('C:/Users/gy21jw/Coding/transfer_resubmission/districts.dill', 'wb') as file:
    dill.dump(districts, file)

# SUBDISTRICTS

In [ ]:
columns = ['p_deg']
titles = ['Partition - Angles',]

sub_districts = districts
nlabels = max([len(sub_districts[column].unique()) for column in columns])
cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
# fig = ci.plot_grid_gdf_columns(sub_districts, columns = columns, titles = titles, geometry_size = 1.5, cmap = cmap, black_background = True,
#                   legend = False, figsize = (15, 10), ncols = 3, nrows = 1)


for n, column in enumerate(columns):
    cmap = ci.rand_cmap(nlabels = len(sub_districts[column].unique()), type_color='bright')
    partitions = ci.polygonise_partitions(sub_districts, column, convex_hull = False)
    ci.plot_gdf(partitions, column = column, cmap = cmap, title =  titles[n], black_background = True,  figsize = (7, 5),
               base_map_gdf = sub_districts, base_map_color = 'grey', base_map_zorder = 1)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import numpy as np

metrics = [
'mean_node_degree',
'dead_end_density',
'intersection_density',
'edge_degree_variance',
'circuity',
'orientation_entropy',
'mean_node_constraint',
'edge_density'
]

# Aggregate metrics by partition
sub_partition_metrics = sub_districts.groupby('p_deg')[metrics].mean()

scaler = StandardScaler()
X = scaler.fit_transform(sub_partition_metrics.values)

scores = {
    'inertia': [],
    'silhouette': [],
    'calinski_harabasz': [],
    'davies_bouldin': []
}
k_range = range(2, 21)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=1977)
    labels = kmeans.fit_predict(X)
    scores['inertia'].append(kmeans.inertia_)
    scores['silhouette'].append(silhouette_score(X, labels))
    scores['calinski_harabasz'].append(calinski_harabasz_score(X, labels))
    scores['davies_bouldin'].append(davies_bouldin_score(X, labels))

scores_df = pd.DataFrame(scores, index=k_range)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0, 0].plot(k_range, scores_df['inertia'], marker='.', linewidth=1)
axes[0, 0].set_xlabel('Number of clusters (k)')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method: Inertia vs k')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(k_range, scores_df['silhouette'], marker='.', linewidth=1, color='green')
axes[0, 1].set_xlabel('Number of clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score vs k (higher is better)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(k_range, scores_df['calinski_harabasz'], marker='.', linewidth=1, color='orange')
axes[1, 0].set_xlabel('Number of clusters (k)')
axes[1, 0].set_ylabel('Calinski-Harabasz Score')
axes[1, 0].set_title('Calinski-Harabasz Score vs k (higher is better)')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(k_range, scores_df['davies_bouldin'], marker='.', linewidth=1, color='red')
axes[1, 1].set_xlabel('Number of clusters (k)')
axes[1, 1].set_ylabel('Davies-Bouldin Score')
axes[1, 1].set_title('Davies-Bouldin Score vs k (lower is better)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

scores_df

In [179]:
from sklearn.cluster import KMeans

# Use the partition_metrics DataFrame from previous step
optimal_k = 14  # Replace with chosen k

kmeans = KMeans(n_clusters=optimal_k, random_state=1977)
sub_partition_metrics['cluster'] = kmeans.fit_predict(sub_partition_metrics)

# Drop existing 'cluster' column if present
if 'cluster' in sub_districts.columns:
    sub_districts = sub_districts.drop(columns=['cluster'])

# Merge cluster labels back to the original districts DataFrame
sub_districts = sub_districts.merge(
    sub_partition_metrics['cluster'],
    left_on='p_deg',
    right_index=True,
    how='left'
)

# Now each edge in districts has its partition's cluster label
print(sub_districts[['p_deg', 'cluster']].head(5))

   p_deg  cluster
1      0       12
2      0       12
3      0       12
4      1        0
5      1        0


In [165]:
sub_districts.columns

Index(['edgeID', 'u', 'v', 'key', 'geometry', 'length', 'name', 'highway',
       'oneway', 'lanes', 'bridge', 'tunnel', 'mean_node_degree',
       'dead_end_density', 'intersection_density', 'edge_degree_variance',
       'circuity', 'orientation', 'orientation_entropy',
       'mean_node_constraint', 'edge_density', 'p_deg', 'taxonomy',
       'dbscan_cluster', 'agg_cluster', 'cluster'],
      dtype='object')

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

columns = ['cluster']
titles = [f'CityImage Partitions by Degree (n={len(partition_counts)})']

nlabels = max([len(sub_districts[column].unique()) for column in columns])

cmap = 'tab20'
# cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
plt.rcParams['figure.dpi'] = 600
fig, ax = plt.subplots(figsize=(10, 8), dpi=800)

sub_districts.to_crs(epsg=3857).plot(
    ax=ax,
    column=columns[0],
    cmap=cmap,
    legend=False,
    linewidth=0.2
)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title(titles[0], fontsize=16)
plt.tight_layout()
plt.show()

---

In [161]:
sub_districts.shape

(120565, 26)

## Prepare metrics for scaling
For a new study area, use the loaded metrics list to select columns from your DataFrame.

## Using Leeds-trained models and scaler
When working with a new study area, load the saved Leeds scaler, kmeans model, metrics list, and taxonomy map before processing the data. This ensures transferability and consistency.

In [ ]:
import dill
import json

# Load the fitted scaler
with open('leeds_scaler.dill', 'rb') as f:
    scaler = dill.load(f)

# Load the fitted KMeans model
with open('leeds_kmeans.dill', 'rb') as f:
    kmeans = dill.load(f)

# Load the taxonomy mapping
with open('leeds_taxonomy_map.json', 'r') as f:
    taxonomy_map = json.load(f)

# Load the metrics list
with open('leeds_metrics.json', 'r') as f:
    metrics = json.load(f)

### Cluster the Partitions

### If using the saved model

In [ ]:
# 1. Aggregate metrics by partition (e.g., 'p_rad')
partition_metrics = districts.groupby('p_rad')[metrics].mean()

# 2. Scale using Leeds scaler
X_scaled = scaler.transform(partition_metrics.values)

# 3. Predict clusters for each partition
partition_metrics['cluster'] = kmeans.predict(X_scaled)

# 4. Merge cluster labels back to the original districts DataFrame
districts = districts.merge(partition_metrics['cluster'], left_on='p_rad', right_index=True, how='left', suffixes=('', '_partition'))

# 5. Apply taxonomy mapping
districts['taxonomy'] = districts['cluster'].map(lambda x: taxonomy_map[str(x)] if str(x) in taxonomy_map else None)

In [ ]:
districts.groupby('taxonomy').count()

In [ ]:
print("Number of unique partitions:", len(partition_metrics))
print("Unique cluster labels assigned:", partition_metrics['cluster'].unique())
print("Value counts:\n", partition_metrics['cluster'].value_counts())
print("Metrics used for Leeds model:", metrics)
print("Columns in partition_metrics:", partition_metrics.columns.tolist())
print(partition_metrics.head())
print(partition_metrics.describe())
print(partition_metrics.nunique())

In [ ]:
import pandas as pd

# Get scaled data as DataFrame for inspection
X_scaled = scaler.transform(partition_metrics[metrics].values)
scaled_df = pd.DataFrame(X_scaled, columns=metrics, index=partition_metrics.index)
print(scaled_df.describe())
print(scaled_df.head())

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

columns = ['cluster']
titles = [f'CityImage Partitions by Radian (n={len(partition_counts)})']

nlabels = max([len(districts[column].unique()) for column in columns])

cmap = 'tab20'
# cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
plt.rcParams['figure.dpi'] = 600
fig, ax = plt.subplots(figsize=(10, 8), dpi=600)

districts.to_crs(epsg=3857).plot(
    ax=ax,
    column=columns[0],
    cmap=cmap,
    legend=False,
    linewidth=0.2
)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
plt.tight_layout()
plt.show()

### 1st time clustering

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import numpy as np

metrics = [
'mean_node_degree',
'dead_end_density',
'intersection_density',
'edge_degree_variance',
'circuity',
'orientation_entropy',
'mean_node_constraint',
'edge_density'
]

# Aggregate metrics by partition
partition_metrics = districts.groupby('p_deg')[metrics].mean()

scaler = StandardScaler()
X = scaler.fit_transform(partition_metrics.values)

scores = {
    'inertia': [],
    'silhouette': [],
    'calinski_harabasz': [],
    'davies_bouldin': []
}
k_range = range(2, 21)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=1977)
    labels = kmeans.fit_predict(X)
    scores['inertia'].append(kmeans.inertia_)
    scores['silhouette'].append(silhouette_score(X, labels))
    scores['calinski_harabasz'].append(calinski_harabasz_score(X, labels))
    scores['davies_bouldin'].append(davies_bouldin_score(X, labels))

scores_df = pd.DataFrame(scores, index=k_range)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0, 0].plot(k_range, scores_df['inertia'], marker='.', linewidth=1)
axes[0, 0].set_xlabel('Number of clusters (k)')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method: Inertia vs k')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(k_range, scores_df['silhouette'], marker='.', linewidth=1, color='green')
axes[0, 1].set_xlabel('Number of clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score vs k (higher is better)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(k_range, scores_df['calinski_harabasz'], marker='.', linewidth=1, color='orange')
axes[1, 0].set_xlabel('Number of clusters (k)')
axes[1, 0].set_ylabel('Calinski-Harabasz Score')
axes[1, 0].set_title('Calinski-Harabasz Score vs k (higher is better)')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(k_range, scores_df['davies_bouldin'], marker='.', linewidth=1, color='red')
axes[1, 1].set_xlabel('Number of clusters (k)')
axes[1, 1].set_ylabel('Davies-Bouldin Score')
axes[1, 1].set_title('Davies-Bouldin Score vs k (lower is better)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

scores_df

In [ ]:
print(pd.DataFrame(X, columns=metrics).describe())

In [182]:
from sklearn.cluster import KMeans

# Use the partition_metrics DataFrame from previous step
optimal_k = 15  # Replace with chosen k

kmeans = KMeans(n_clusters=optimal_k, random_state=1977)
partition_metrics['cluster'] = kmeans.fit_predict(partition_metrics)

# Drop existing 'cluster' column if present
if 'cluster' in districts.columns:
    districts = districts.drop(columns=['cluster'])

# Merge cluster labels back to the original districts DataFrame
districts = districts.merge(
    partition_metrics['cluster'],
    left_on='p_deg',
    right_index=True,
    how='left'
)

# Now each edge in districts has its partition's cluster label
print(districts[['p_deg', 'cluster']].head())

   p_deg  cluster
1      0        0
2      0        0
3      0        0
4      1       14
5      1       14


---

### DBSCAN attempt

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

metrics = [
    'mean_node_degree',
    'dead_end_density',
    'intersection_density',
    'edge_degree_variance',
    'circuity',
    'orientation_entropy',
    'edge_density'
]

# Aggregate metrics by partition
partition_metrics = districts.groupby('p_deg')[metrics].mean()

# Scale the data
scaler = StandardScaler()
X = scaler.fit_transform(partition_metrics.values)


eps=0.5
min_samples=1
# Run DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples)  # Tune as needed
labels = dbscan.fit_predict(X)

# Add cluster labels to partition_metrics
partition_metrics['dbscan_cluster'] = labels

# Remove any existing dbscan_cluster columns before merging
for col in ['dbscan_cluster', 'dbscan_cluster_x', 'dbscan_cluster_y']:
    if col in districts.columns:
        districts = districts.drop(columns=[col])

# Merge new DBSCAN cluster labels
districts = districts.merge(
    partition_metrics[['dbscan_cluster']],
    left_on='p_deg',
    right_index=True,
    how='left'
)


# Check results
print("eps: ",eps)
print("min_samples: ",min_samples)
print(districts['dbscan_cluster'].value_counts())

eps:  0.8
min_samples:  1
dbscan_cluster
0     81093
14     1729
26     1514
35     1485
33     1475
      ...  
71        7
78        7
82        4
13        4
7         3
Name: count, Length: 83, dtype: int64


In [222]:
districts.columns

Index(['edgeID', 'u', 'v', 'key', 'geometry', 'length', 'name', 'highway',
       'oneway', 'lanes', 'bridge', 'tunnel', 'mean_node_degree',
       'dead_end_density', 'intersection_density', 'edge_degree_variance',
       'circuity', 'orientation', 'orientation_entropy',
       'mean_node_constraint', 'edge_density', 'p_deg', 'taxonomy',
       'dbscan_cluster', 'agg_cluster', 'cluster', 'mode_buildingage_period'],
      dtype='object')

In [ ]:
# import matplotlib.pyplot as plt
# import contextily as ctx

# columns = ['dbscan_cluster']
# titles = [f'CityImage Partitions by Degree (n={len(partition_counts)})']

# nlabels = max([len(districts[column].unique()) for column in columns])

# cmap = 'tab20'
# # cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
# plt.rcParams['figure.dpi'] = 600
# fig, ax = plt.subplots(figsize=(10, 8), dpi=600)

# districts.to_crs(epsg=3857).plot(
#     ax=ax,
#     column=columns[0],
#     cmap=cmap,
#     legend=False,
#     linewidth=0.2
# )
# ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
# plt.tight_layout()
# plt.show()

---

### hierarchical clustering

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import AgglomerativeClustering

# metrics = [
#     'mean_node_degree',
#     'dead_end_density',
#     'intersection_density',
#     'edge_degree_variance',
#     'circuity',
#     'orientation_entropy',
#     'edge_density'
# ]

# # Aggregate metrics by partition
# partition_metrics = districts.groupby('p_deg')[metrics].mean()

# # Scale the data
# scaler = StandardScaler()
# X = scaler.fit_transform(partition_metrics.values)

# # Run Agglomerative Clustering
# n_clusters = 20  # Change this to your desired number of clusters
# agg = AgglomerativeClustering(
#     n_clusters=None,
#     linkage='ward',
#     metric="euclidean",
#     distance_threshold=5
#     )
# labels = agg.fit_predict(X)

# # Add cluster labels to partition_metrics
# partition_metrics['agg_cluster'] = labels

# # Merge cluster labels into districts
# for col in ['agg_cluster', 'agg_cluster_x', 'agg_cluster_y']:
#     if col in districts.columns:
#         districts = districts.drop(columns=[col])

# districts = districts.merge(
#     partition_metrics[['agg_cluster']],
#     left_on='p_deg',
#     right_index=True,
#     how='left'
# )

# # Check results
# print(districts['agg_cluster'].value_counts())

agg_cluster
5     17748
0     16962
1     15825
7     12024
2     11686
4      7989
3      7930
8      7478
9      6747
17     5719
13     3843
14     3547
12     1935
10     1086
6        28
15        7
16        7
11        4
Name: count, dtype: int64


In [ ]:
# import matplotlib.pyplot as plt
# import contextily as ctx

# columns = ['dbscan_cluster']
# titles = [f'CityImage Partitions by Degree (n={len(partition_counts)})']

# nlabels = max([len(districts[column].unique()) for column in columns])

# cmap = 'tab20'
# # cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
# plt.rcParams['figure.dpi'] = 600
# fig, ax = plt.subplots(figsize=(10, 8), dpi=600)

# districts.to_crs(epsg=3857).plot(
#     ax=ax,
#     column=columns[0],
#     cmap=cmap,
#     legend=False,
#     linewidth=0.2
# )
# ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
# plt.tight_layout()
# plt.show()

---

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

columns = ['cluster']
titles = [f'CityImage Partitions by Degree (n={len(partition_counts)})']

nlabels = max([len(districts[column].unique()) for column in columns])

cmap = 'tab20'
# cmap = ci.rand_cmap(nlabels = nlabels, type_color='bright')
plt.rcParams['figure.dpi'] = 600
fig, ax = plt.subplots(figsize=(10, 8), dpi=600)

districts.to_crs(epsg=3857).plot(
    ax=ax,
    column=columns[0],
    cmap=cmap,
    legend=False,
    linewidth=0.2
)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title(titles[0], fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# # remove cluster_x and cluster_y from districts
# districts = districts.drop(columns=['cluster_x', 'cluster_y'])
# districts

In [187]:
# To get all data grouped by cluster:
for cluster_id in sorted(districts['cluster'].unique()):
    print(f"\nCluster {cluster_id} data:")
    print(districts[districts['cluster'] == cluster_id])


Cluster 0 data:
        edgeID      u      v  key  \
1            1      1  94752    0   
2            2      1  11080    0   
3            3      1  11041    0   
20          20      7  85644    0   
21          21      7  79205    0   
...        ...    ...    ...  ...   
253690  253690  98614  98615    1   
253691  253691  98615  98619    0   
253694  253694  98615  98616    0   
253696  253696  98616  98617    1   
254459  254459  98899  98900    0   

                                                 geometry  \
1       LINESTRING (-194159.136 6003767.155, -194149.7...   
2       LINESTRING (-194159.136 6003767.155, -194167.5...   
3       LINESTRING (-194159.136 6003767.155, -194157.2...   
20      LINESTRING (-185109.601 6005100.218, -185124.2...   
21      LINESTRING (-185109.601 6005100.218, -185112.3...   
...                                                   ...   
253690  LINESTRING (-199914.724 6008196.596, -199909.4...   
253691  LINESTRING (-199887.979 6008171.956, -1998

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

districts['cluster'] = districts['cluster'].astype(str)
melted = districts.melt(id_vars='cluster', value_vars=metrics)
melted['cluster'] = melted['cluster'].astype(int)

plt.figure(figsize=(14, 6))
cluster_order = sorted(melted['cluster'].unique(), reverse=True)
palette = sns.color_palette("tab20", n_colors=len(cluster_order))
sns.boxplot(x='variable', y='value', hue='cluster', data=melted, palette=palette, hue_order=cluster_order)

plt.title('Metric Distributions by Cluster')
plt.xlabel('Metric')
plt.ylabel('Value')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

### Cluster Profiles

In [189]:
# List of metrics used for clustering
metrics = [
    'mean_node_degree',
    'dead_end_density',
    'intersection_density',
    'edge_degree_variance',
    'circuity',
    'orientation_entropy',
    'mean_node_constraint',
    'edge_density'
]

# Group by cluster and calculate summary statistics
cluster_profiles = partition_metrics.groupby('cluster')[metrics].agg(['mean', 'median', 'min', 'max', 'std'])

In [ ]:
cluster_profiles

In [ ]:
# Show mean metric values for each cluster
metrics = [
    'mean_node_degree',
    'dead_end_density',
    'intersection_density',
    'edge_degree_variance',
    'circuity',
    'orientation_entropy',
    'mean_node_constraint',
    'edge_density'
]

partition_metrics.groupby('cluster')[metrics].mean()

In [191]:
# Convert 'cluster' to int
districts['cluster'] = districts['cluster'].astype(int)

In [ ]:
import matplotlib.pyplot as plt
cn = 14
cluster_edges = districts[districts['cluster'] == cn]
fig, ax = plt.subplots(figsize=(10, 8), dpi=600)
cluster_edges.plot(ax=ax, color="#1f77b4", linewidth=0.5, alpha=0.7)
ax.set_title(f"Cluster {cn} Edges")
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [46]:
taxonomy_map = {
    0: "Organic",
    1: "Mixed",
    2: "Tree-like",
    3: "Regular-grid",
    4: "Mixed",
    5: "Mixed",
    6: "Mixed",
    7: "Tree-like",
    8: "Regular-grid",
    9: "Deformed-grid",
    10: "Mixed",
    11: "Deformed-grid",
    12: "Organic",
    13: "Organic",
    14: "Mixed"
}

In [47]:
# Add taxonomy to districts DataFrame
districts['taxonomy'] = districts['cluster'].astype(int).map(taxonomy_map)

result = districts.groupby('taxonomy').agg({
    'cluster': [lambda x: sorted(set(x)), 'count']
}).reset_index()

# Rename columns for clarity
result.columns = ['taxonomy', 'clusters', 'count']
print(result)




        taxonomy              clusters  count
0  Deformed-grid               [9, 11]   8362
1          Mixed  [1, 4, 5, 6, 10, 14]  62554
2        Organic           [0, 12, 13]  24589
3   Regular-grid                [3, 8]   4768
4      Tree-like                [2, 7]  20292


In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd

# List of unique taxonomy types (edit if you want to group similar types)
taxonomy_types = districts['taxonomy'].unique()

# Set up plot
fig, axes = plt.subplots(2, 3, figsize=(18, 12), dpi=600)
axes = axes.flatten()

for i, tax in enumerate(taxonomy_types):
    ax = axes[i]
    subset = districts[districts['taxonomy'] == tax].to_crs(epsg=3857)
    subset.plot(ax=ax, color="#03375c", linewidth=0.2, alpha=1)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    ax.set_title(tax, fontsize=16)
    ax.set_axis_off()

# Remove unused axes if less than 6 types
for j in range(len(taxonomy_types), 6):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [195]:
districts.columns

Index(['edgeID', 'u', 'v', 'key', 'geometry', 'length', 'name', 'highway',
       'oneway', 'lanes', 'bridge', 'tunnel', 'mean_node_degree',
       'dead_end_density', 'intersection_density', 'edge_degree_variance',
       'circuity', 'orientation', 'orientation_entropy',
       'mean_node_constraint', 'edge_density', 'p_deg', 'taxonomy',
       'dbscan_cluster', 'agg_cluster', 'cluster'],
      dtype='object')

In [ ]:
# List of metrics used for clustering
metrics = [
    'mean_node_degree',
    'dead_end_density',
    'intersection_density',
    'edge_degree_variance',
    'circuity',
    'orientation_entropy',
    'mean_node_constraint',
    'edge_density'
]

# Group by cluster and calculate summary statistics
cluster_profiles = partition_metrics.groupby('cluster')[metrics].agg(['mean', 'median', 'min', 'max', 'std'])

# Display the profile for each cluster
display(cluster_profiles)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract mean values for each metric and cluster
mean_profiles = cluster_profiles.xs('mean', axis=1, level=1)

plt.figure(figsize=(12, 6))
sns.heatmap(mean_profiles, annot=True, fmt=".2f", cmap="viridis", cbar_kws={'label': 'Mean Value'})
plt.title("Cluster Profiles: Mean Metric Values per Cluster")
plt.xlabel("Metric")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()

---

---

### Save the Leeds model for use in other study areas 

In [56]:
import dill
import json

# Save the fitted StandardScaler
with open("leeds_scaler.dill", "wb") as f:
    dill.dump(scaler, f)

# Save the fitted KMeans clustering model
with open("leeds_kmeans.dill", "wb") as f:
    dill.dump(kmeans, f)

# Save the taxonomy mapping dictionary
taxonomy_map = {
    0: "Organic",
    1: "Mixed",
    2: "Tree-like",
    3: "Regular-grid",
    4: "Mixed",
    5: "Mixed",
    6: "Mixed",
    7: "Tree-like",
    8: "Regular-grid",
    9: "Deformed-grid",
    10: "Mixed",
    11: "Deformed-grid",
    12: "Organic",
    13: "Organic",
    14: "Mixed"
}
with open("leeds_taxonomy_map.json", "w") as f:
    json.dump(taxonomy_map, f)

# Save the list of metrics used for clustering
metrics = [
    'mean_node_degree',
    'dead_end_density',
    'intersection_density',
    'edge_degree_variance',
    'circuity',
    'orientation_entropy',
    'mean_node_constraint',
    'edge_density'
]
with open("leeds_metrics.json", "w") as f:
    json.dump(metrics, f)

with open("leeds_cluster_profiles.dill", "wb") as f:
          dill.dump(cluster_profiles, f)

with open("partition_metrics.dill", "wb") as f:
          dill.dump(partition_metrics, f)

---

### LOAD THE LEEDS MODEL SCALER AND KMEANS DATA ETC

In [ ]:
import dill
import json

# Load the fitted scaler
with open('leeds_scaler.dill', 'rb') as f:
    scaler = dill.load(f)

# Load the fitted KMeans model
with open('leeds_kmeans.dill', 'rb') as f:
    kmeans = dill.load(f)

# Load the taxonomy mapping
with open('leeds_taxonomy_map.json', 'r') as f:
    taxonomy_map = json.load(f)

# Load the metrics list
with open('leeds_metrics.json', 'r') as f:
    metrics = json.load(f)

---

---

### Save the cluster profiles from Leeds for use in the new data

In [ ]:
import numpy as np
import pandas as pd

# Leeds cluster profiles (mean values for each metric)
# leeds_profiles: DataFrame, index=Leeds cluster number, columns=metrics
# new_profiles: DataFrame, index=New cluster number, columns=metrics

def match_clusters_by_profile(leeds_profiles, new_profiles, metrics):
    mapping = {}
    for new_cluster, new_row in new_profiles.iterrows():
        # Compute distances to all Leeds clusters
        distances = leeds_profiles[metrics].apply(
            lambda leeds_row: np.linalg.norm(leeds_row.values - new_row[metrics].values), axis=1
        )
        # Find the Leeds cluster with the smallest distance
        best_leeds_cluster = distances.idxmin()
        mapping[new_cluster] = best_leeds_cluster
    return mapping

# Example usage:
# mapping = match_clusters_by_profile(leeds_profiles, new_profiles, metrics)
# new_districts['leeds_cluster'] = new_districts['cluster'].map(mapping)
# new_districts['taxonomy'] = new_districts['leeds_cluster'].map(leeds_taxonomy_map)

---

---

### NGD Buildings

In [70]:
# Load the NGD Buildings GPKG
path_buildings = (r"C:\Users\gy21jw\Coding\transfer_resubmission\bld_fts_building.gpkg")

leeds_buildings = gpd.read_file(
    path_buildings)

del path_buildings

In [ ]:
leeds_buildings.columns

In [196]:
partitions.head(2)

,geometry,p_deg
0,"POLYGON ((-196978.894 6003950.383, -196990.272...",0
1,"POLYGON ((-188207.485 6004214.919, -188205.328...",1


In [ ]:
# Filter the data so we are keeping the following columns "buildingage_period", "buildingage_year" and "geometry" only

leeds_buildings_clean = leeds_buildings[["osid", "buildingage_period", "buildingage_year", "geometry"]]
leeds_buildings_clean

In [ ]:
columns = ['p_deg']
titles = ['Partition - Degree Angles']
for n, column in enumerate(columns):
    cmap = ci.rand_cmap(nlabels = len(districts[column].unique()), type_color='bright')
    polygonised_partitions = ci.polygonise_partitions(districts, column, convex_hull = True, buffer=0)
    fig, ax = plt.subplots(figsize=(15, 10), dpi=600)
    polygonised_partitions.to_crs(epsg=3857).plot(
        ax=ax,
        column=column,
        cmap=cmap,
        legend=False,
        linewidth=1.5
    )
    ax.set_title(titles[0], fontsize=16)
    plt.tight_layout()
    plt.show()

In [207]:
polygonised_partitions.head(2)

,geometry,p_deg
0,"POLYGON ((-196027.577 6002538.165, -196032.522...",0
1,"POLYGON ((-184686.944 6003279.864, -187224.999...",1


In [210]:
import geopandas as gpd
from scipy.stats import mode

# Ensure both GeoDataFrames use the same CRS
leeds_buildings_clean = leeds_buildings_clean.to_crs(polygonised_partitions.crs)

# Spatial join: assign buildings to partitions
joined = gpd.sjoin(leeds_buildings_clean, polygonised_partitions, how='left', predicate='within')

# Group by partition and get the mode (most common) buildingage_period
def get_mode(series):
    mode = series.mode()
    if not mode.empty:
        return mode.iloc[0]
    else:
        return None

mode_buildingage = joined.groupby('index_right')['buildingage_period'].agg(get_mode)
polygonised_partitions['mode_buildingage_period'] = polygonised_partitions.index.map(mode_buildingage)

# Check results
print(polygonised_partitions[['p_deg', 'mode_buildingage_period']])

     p_deg mode_buildingage_period
0        0               1945-1959
1        1               1919-1944
2        2               2010-2019
3        3               1945-1959
4        4               1990-1999
..     ...                     ...
249    253               1980-1989
250     18                     NaN
251    245                     NaN
252    248                     NaN
253     33               1960-1979

[254 rows x 2 columns]


In [211]:
polygonised_partitions.head(2)

,geometry,p_deg,mode_buildingage_period
0,"POLYGON ((-196027.577 6002538.165, -196032.522...",0,1945-1959
1,"POLYGON ((-184686.944 6003279.864, -187224.999...",1,1919-1944


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10), dpi=600)
polygonised_partitions.to_crs(epsg=3857).plot(
        ax=ax,
        column='mode_buildingage_period',
        cmap='tab20',
        legend=True,
        linewidth=1.5,
        alpha=0.5
    )
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title('average build age ranges', fontsize=16)
plt.tight_layout()
plt.show()

In [218]:
districts = districts.merge(
    mode_buildingage.rename('mode_buildingage_period'),
    left_on='p_deg',
    right_index=True,
    how='left'
)

In [220]:
districts.head(2)

,edgeID,u,v,key,geometry,length,name,highway,oneway,lanes,...,orientation,orientation_entropy,mean_node_constraint,edge_density,p_deg,taxonomy,dbscan_cluster,agg_cluster,cluster,mode_buildingage_period
1,1,1,94752,0,"LINESTRING (-194159.136 6003767.155, -194149.7...",16.52,Royston Hill,primary,False,2,...,124.54,2.67,0.51,39.75,0,Organic,0,2,0,1945-1959
2,2,1,11080,0,"LINESTRING (-194159.136 6003767.155, -194167.5...",27.31,Royston Hill,primary,False,2,...,108.01,2.68,0.42,40.69,0,Organic,0,2,0,1945-1959


---